In [ ]:
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

In [ ]:
spark = SparkSession.builder.appName("ETL Local").getOrCreate()

In [ ]:
dataframe_industrial = spark.read.csv("./data/industrial_monitoring_logs.csv", header=True, inferSchema=True)

In [ ]:
dataframe_industrial_health = dataframe_industrial.withColumn(
    "health_category",
    when(col("vibration") < 1, "Normal")
    .when(col("vibration") < 2, "Warning")
    .otherwise("Critical")
)

In [ ]:
dataframe_equipment_health = dataframe_industrial_health.groupBy(["equipment", "health_category"]).agg(
    {
        "temperature": "mean",
        "pressure": "mean",
        "vibration": "mean",
        "humidity": "mean"
    }
)

In [ ]:
os.makedirs("./data/output", exist_ok=True)
dataframe_equipment_health.write.mode("overwrite").parquet("./data/output/equipment_health.parquet")

In [ ]:
read_parquet = pd.read_parquet("./data/output/equipment_health.parquet/", engine="pyarrow")
read_parquet